### 연습문제 1

Pima 인디언 당뇨 분석 데이터에 대하여
- 5–fold 교차 검증을 수행하시오
- 모델은 자유롭게 2가지를 선택하고,
- 데이터 스케일링을 하시오

In [7]:
import pandas as pd
import os

path = r'D:\elice_python\GAS_5\pytest_machine'
os.chdir(path)

In [8]:
data = pd.read_csv('pima-indians-diabetes.csv')

In [9]:
print(data.head())
print(data.shape)
print(data.info())

   pregnant  plasma  pressure  thickness  insulin   bmi  pedigree  age  \
0         6     148        72         35        0  33.6     0.627   50   
1         1      85        66         29        0  26.6     0.351   31   
2         8     183        64          0        0  23.3     0.672   32   
3         1      89        66         23       94  28.1     0.167   21   
4         0     137        40         35      168  43.1     2.288   33   

   diabetes  
0         1  
1         0  
2         1  
3         0  
4         1  
(768, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pregnant   768 non-null    int64  
 1   plasma     768 non-null    int64  
 2   pressure   768 non-null    int64  
 3   thickness  768 non-null    int64  
 4   insulin    768 non-null    int64  
 5   bmi        768 non-null    float64
 6   pedigree   768 non-null    float6

In [10]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

print(X.shape, y.shape)

(768, 8) (768,)


In [11]:
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify= y, random_state=42)
skf = StratifiedKFold(n_splits=5, shuffle=True)

In [12]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
mms = MinMaxScaler()
X_train_mms = mms.fit_transform(X_train)
X_test_mms = mms.transform(X_test)

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
logreg = LogisticRegression(solver='saga', penalty='elasticnet', C=5.0, l1_ratio=0.1, max_iter=10000)
svc = SVC(kernel='rbf', C=5.0, gamma=0.2)

In [24]:
score_svc_mms = cross_val_score(svc, X, y, cv=skf)
print(score_svc_mms,'\n', score_svc_mms.mean())

[0.64935065 0.64935065 0.64935065 0.65359477 0.65359477] 
 0.6510482981071216


In [25]:
score_logleg_mms = cross_val_score(logreg, X, y, cv=skf)
print(score_logleg_mms,'\n', score_logleg_mms.mean())

[0.70779221 0.72727273 0.63636364 0.65359477 0.76470588] 
 0.6979458450046685


### 연습문제 2
- 위에서 선택한 모델들로 GridSearchCV를 수행하시오
- 사이킷런 공식 문서를 보고 적용할 수 있는 파라미터를 찾으시오
- 찾은 최적의 값과 스케일된 데이터로 다시 모델을 만들어
- 테스트 데이터 정확도를 각각 산출하시오
- 또한, .predict()를 이용하여 테스트 데이터에 대한 예측 결과를 도출하시오

In [17]:
from sklearn.model_selection import GridSearchCV
import numpy as np
arange = np.concatenate((
    np.arange(0.001, 0.01, 0.001),
    np.arange(0.01, 0.1, 0.01),
    np.arange(0.1, 1, 0.1),
    np.arange(1, 10, 1),
    np.arange(10, 100, 10)
))

In [19]:
param_grid_svc = {'C':arange,
                  'gamma':arange,
                  'random_state':[42]}

gsvc = GridSearchCV(SVC(), param_grid=param_grid_svc, cv = 5)
gsvc.fit(X_train_mms,y_train)

print(gsvc.best_score_)
print(gsvc.best_params_)

0.79167916041979
{'C': 20.0, 'gamma': 0.060000000000000005, 'random_state': 42}


In [29]:
param_grid_logleg = {
    'l1_ratio': np.arange(0.0,1,0.05),
    'C': arange,
    'solver' : ['saga'],
    'penalty':['elasticnet'],
    'max_iter':[10000]
}

glogreg = GridSearchCV(LogisticRegression(), param_grid=param_grid_logleg, cv = 5)
glogreg.fit(X_train_mms,y_train)

print(glogreg.best_score_)
print(glogreg.best_params_)

0.7916941529235382
{'C': 6.0, 'l1_ratio': 0.0, 'max_iter': 10000, 'penalty': 'elasticnet', 'solver': 'saga'}


### 연습문제 3
앞의 결과를 바탕으로
- 두 모델 각각에 대하여 혼동행렬(오차행렬)을 구하시오
- 또한, Precision, Recall, F1 score를 구하시오

In [31]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
prediction_gsvc= gsvc.predict(X_test_mms)
prediction_glogleg = glogreg.predict(X_test_mms)

In [40]:
def cprf(y_true, y_pred):
    confusion = confusion_matrix(y_true, y_pred,)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(' confusion_matrix : \n', confusion)
    print(' precision : ', precision)
    print(' recall : ', recall)
    print(' f1 score : ', f1)
    print()

In [41]:
cprf(y_test, prediction_gsvc)

 confusion_matrix : 
 [[108  17]
 [ 35  32]]
 precision :  0.6530612244897959
 recall :  0.47761194029850745
 f1 score :  0.5517241379310345



In [42]:
cprf(y_test, prediction_glogleg)

 confusion_matrix : 
 [[107  18]
 [ 33  34]]
 precision :  0.6538461538461539
 recall :  0.5074626865671642
 f1 score :  0.5714285714285714



### 연습문제 4
- 앞에서 만든 모델로 ROC AUC score를 구하시오
- 0과 1의 확률이 나오는 모델을 대상으로 ROC 커브를 그리시오